In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/NLP_and_CV_Projects/NLP/Hung')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Mon Jan 25 03:07:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.8MB 14.8MB/s 
     |████████████████████████████████| 890kB 53.2MB/s 
     |████████████████████████████████| 2.9MB 42.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5824a149c8e0c8f0bd1a4963f742752d1c760c2d15b3cf7b32d28d9b9d3fe857
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

#phobert = AutoModel.from_pretrained("vinai/phobert-base")
custokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


In [ ]:
# train_path = 'VNTQcorpus-big.txt'
# test_path = 'VNTQcorpus-small.txt'

In [ ]:
# import re
# import pandas as pd

# with open(train_path) as f:
#     train = f.read()
# with open(test_path) as f:
#     test = f.read()

In [ ]:
# def clean_text_files(data, dest_path):
#     f = open(dest_path, 'w')
#     data_write = re.sub("[0-9\.,\(\)\?\:\\'\!\"\-]", '', data)
#     f.write(data_write)

In [ ]:
# clean_text_files(train,'VNTQcorpus-big-cleaned.txt')
# clean_text_files(test,'VNTQcorpus-small-cleaned.txt')

# print("Train dataset length: "+str(len(train)))
# print("Test dataset length: "+ str(len(test)))

In [ ]:
train_path = 'VNTQcorpus-big-cleaned.txt'
test_path = 'VNTQcorpus-small-cleaned.txt'

In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path, test_path, tokenizer):
    train_dataset = TextDataset(
          tokenizer=custokenizer,
          file_path=train_path,
          block_size=256)
     
    test_dataset = TextDataset(
          tokenizer=custokenizer,
          file_path=test_path,
          block_size=256)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=custokenizer, mlm=False,
    )
    return train_dataset,test_dataset,data_collator

train_dataset,test_dataset,data_collator = load_dataset(train_path,test_path,custokenizer)

/usr/local/lib/python3.6/dist-packages/transformers/data/datasets/language_modeling.py:58: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [ ]:
print(custokenizer.decode(train_dataset[1]))

<s> nhưng bị Diệp Đại Thúy nắm chặt cánh tay bèn hỏi Đại Thúy sao nàng làm thế Diệp Đại Thúy dịu dàng nhìn chàng nhỏ nhẹ nói Tướng công thiếp biết có chuyện gì rồi chỉ trách chàng hành sự hấp tấp chưa suy nghĩ đối phương thì quỷ kế đa đoan mấy vị kia lại hành sự lỗ mãng<unk> Lãnh Huyết Diễm Nữ cũng nói Tướng công Diệp cô nương nói đúng lắm<unk> Nếu luận về võ công thì sau khi tướng công luyện xong Lục Giáp chân kinh phải là quán thế<unk> Nhưng giang hồ có rất nhiều phong ba âm hiểm tướng công nên nghe lời Diệp cô nương<unk> Đoạn lấy trong túi ra một chiếc lọ nhỏ màu xanh thỏ thẻ nói Đây là giải dược tướng công cầm lấy đi<unk> Đông Phương Thanh Vân cười ngạo Thành ý tâm lĩnh có điều tại hạ quyết chẳng chịu nhận không ân tình của người đại ân của phu nhân đối với nghĩa mẫu của tại hạ còn chưa kịp báo đáp<unk> Lúc này Diệp Đại Thúy đã hiểu tại sao Đông Phương Thanh Vân trước sau cứ khách khí với nữ nhân kia<unk> Lời của Đông Phương Thanh Vân khiến nàng rất vui<unk> Nàng bèn dịu dàng nói C

In [ ]:
from transformers import Trainer, TrainingArguments, GPT2Config, GPT2LMHeadModel

In [ ]:
# task = {"text-generation": {"do_sample": True, "max_length": 256}} #edit output size

In [ ]:
# configuration = GPT2Config(vocab_size=64002, n_positions=258, n_ctx=258,
#                            task_specific_params=task,
#                            eos_token_id=custokenizer.eos_token_id,
#                            bos_token_id=custokenizer.bos_token_id, 
#                            pad_token_id=custokenizer.pad_token_id,
#                            sep_token_id=custokenizer.sep_token_id
#                            )
# model = GPT2LMHeadModel(configuration)
# model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(64002, 768)
    (wpe): Embedding(258, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Layer

In [ ]:
model = GPT2LMHeadModel.from_pretrained('./VNTQcorpus/checkpoint-60000')

In [ ]:
training_args = TrainingArguments(
    output_dir="VNTQcorpus", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    per_device_train_batch_size=16, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 500, # number of steps between evaluations
    save_steps=12000, # after # steps model is saved 
    save_total_limit = 2, # delete other checkpoints
    warmup_steps=500,
    prediction_loss_only=True,    # number of warmup steps for learning rate scheduler
    )

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    # tb_writer=writer
)

In [ ]:
trainer.train(model_path = './VNTQcorpus/checkpoint-60000')
trainer.save_model()

Step,Training Loss
60500,0.029800
61000,3.648200
61500,3.665300
62000,3.663000
62500,3.668000
63000,3.668900
63500,3.664000
64000,3.668500
64500,3.663400
65000,3.669700


In [ ]:
model.save_pretrained("./VNTQcorpus")